In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from transformers import DPRContextEncoder, DPRContextEncoderTokenizerFast

tokenizer = DPRContextEncoderTokenizerFast.from_pretrained("facebook/dpr-ctx_encoder-multiset-base")
model = DPRContextEncoder.from_pretrained("../temp/dpr/models/ctx_encoder_dpr")

/opt/conda/envs/read/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizerFast'.


In [3]:
import pandas as pd

df = pd.read_json("../data/totto2/filtered/train.jsonl", lines=True)

In [4]:
import json

df["table_values"] = df["table"].apply(lambda x: pd.DataFrame(json.loads(x)).values.flatten().tolist())

In [5]:
import itertools


all_values = list(itertools.chain.from_iterable(df["table_values"].tolist()))

In [6]:
import random

sampled_values = random.sample(all_values, 10000)

In [7]:
import re
from flair.data import Sentence
from flair.models import SequenceTagger

tagger = SequenceTagger.load('ner')

2022-10-13 01:11:07.152770: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/opt/conda/envs/read/lib/python3.8/site-packages/huggingface_hub/file_download.py:621: FutureWarning: `cached_download` is the legacy way to download files from the HF hub, please consider upgrading to `hf_hub_download`
  warnings.warn(
Downloading: 100%|███████████████████████████████████████████████████████████████████████████████| 432M/432M [00:04<00:00, 88.6MB/s]

2022-10-13 01:11:16,440 loading file /root/.flair/models/ner-english/4f4cdab26f24cb98b732b389e6cebc646c36f54cfd6e0b7d3b90b25656e4262f.8baa8ae8795f4df80b28e7f7b61d788ecbb057d1dc85aacb316f1bd02837a4a4


2022-10-13 01:11:18,413 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>


In [8]:
labels = []

for value in sampled_values:
    if not re.search('[A-Za-z]', value):
        labels.append(1)
    else:
        # sentence = Sentence(str(value))
        # tagger.predict(sentence)
        # for entity in sentence.get_spans('ner'):
        #     tag = entity.get_labels("ner")[0].value
        #     labels.append(2)
        #     break
        # else:
        #     labels.append(0)
        labels.append(0)

In [9]:
encodings = []

for value in sampled_values:
    inputs = tokenizer(value, return_tensors="pt", padding=True, truncation=True)
    encoding = model(**inputs).pooler_output.detach().cpu().numpy()
    encodings.append(encoding[0])

In [10]:
import numpy as np
import seaborn as sns
from sklearn.manifold import TSNE

tsne = TSNE(n_components=2, perplexity=10, random_state=42, n_iter=15000, metric="cosine")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [11]:
tsne_result = tsne.fit_transform(np.asarray(encodings))

/opt/conda/envs/read/lib/python3.8/site-packages/sklearn/manifold/_t_sne.py:800: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
/opt/conda/envs/read/lib/python3.8/site-packages/sklearn/manifold/_t_sne.py:810: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(


In [12]:
sns.scatterplot(x=tsne_result[:, 0], y=two_dim[:, 1], c=labels, style=labels)

NameError: name 'two_dim' is not defined

In [ ]:
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(6,6))

ax = Axes3D(fig) # Method 1
sns.scatterplot(x=two_dim[:, 0], y=two_dim[:, 1], c=labels, style=labels)

In [ ]:
two_dim.shape